# Interactive Polygon Drawing Tool

This interactive polygon drawing tool provides:

✅ **Easy polygon drawing** with intuitive map controls  
✅ **Real-time WKT export** for immediate use with phidown  
✅ **Polygon editing and deletion** capabilities  
✅ **WKT import functionality** to visualize existing polygons  
✅ **Multiple basemap options** including satellite imagery  
✅ **Seamless integration** with phidown search functionality  

## Features

- Interactive map with drawing controls
- Real-time coordinate extraction
- WKT format output (compatible with phidown search)
- Clear and reset functionality
- Multiple polygon support

In [1]:
# Import required libraries
from ipyleaflet import (
    Map, 
    DrawControl, 
    GeoJSON, 
    LayersControl,
    basemaps,
    Marker,
    Popup
)
from ipywidgets import (
    VBox, 
    HBox, 
    Button, 
    Output, 
    HTML, 
    Textarea,
    Label,
    Layout
)
import json
from shapely.geometry import Polygon, mapping
from shapely import wkt
from typing import List, Dict, Any, Optional

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Interactive Polygon Drawing Tool Class

Let's create a comprehensive class that handles all the polygon drawing functionality:

In [2]:
class InteractivePolygonTool:
    """
    Interactive polygon drawing tool using ipyleaflet.
    
    This class provides functionality to draw polygons on an interactive map,
    extract coordinates in WKT format, and manage multiple polygons.
    
    Attributes:
        map (Map): The ipyleaflet map widget
        draw_control (DrawControl): Drawing control for the map
        output (Output): Output widget for displaying information
        wkt_output (Textarea): Text area for WKT output
        polygons (List[Dict]): List of drawn polygons with metadata
    """
    
    def __init__(self, center: tuple = (45.0, 0.0), zoom: int = 2, 
                 basemap=basemaps.OpenStreetMap.Mapnik):
        """
        Initialize the interactive polygon tool.
        
        Args:
            center (tuple): Initial map center coordinates (lat, lon)
            zoom (int): Initial zoom level
            basemap: Basemap to use for the map
        """
        self.polygons: List[Dict[str, Any]] = []
        self._setup_map(center, zoom, basemap)
        self._setup_controls()
        self._setup_ui()
    
    def _setup_map(self, center: tuple, zoom: int, basemap) -> None:
        """
        Setup the main map widget.
        
        Args:
            center (tuple): Map center coordinates
            zoom (int): Initial zoom level
            basemap: Basemap to use
        """
        self.map = Map(
            center=center,
            zoom=zoom,
            basemap=basemap,
            scroll_wheel_zoom=True,
            layout=Layout(height='500px', width='100%')
        )
        
        # Add layer control
        layers_control = LayersControl(position='topright')
        self.map.add_control(layers_control)
    
    def _setup_controls(self) -> None:
        """
        Setup drawing controls for the map.
        """
        self.draw_control = DrawControl(
            polygon={
                'shapeOptions': {
                    'fillColor': '#3388ff',
                    'color': '#0000ff',
                    'fillOpacity': 0.3,
                    'weight': 2
                }
            },
            rectangle={
                'shapeOptions': {
                    'fillColor': '#ff3333',
                    'color': '#ff0000',
                    'fillOpacity': 0.3,
                    'weight': 2
                }
            },
            polyline={},
            circle={},
            circlemarker={},
            marker={},
            edit=True,
            remove=True
        )
        
        # Add event handlers using observe method
        self.draw_control.on_draw(self._handle_draw)
        # For edit and delete, we'll use a single observer on the data attribute
        self.draw_control.observe(self._handle_data_change, names=['data'])
        
        self.map.add_control(self.draw_control)
    
    def _setup_ui(self) -> None:
        """
        Setup the user interface widgets.
        """
        # Output widget for messages
        self.output = Output()
        
        # WKT output textarea
        self.wkt_output = Textarea(
            placeholder='WKT coordinates will appear here...',
            description='WKT Output:',
            layout=Layout(height='100px', width='100%'),
            style={'description_width': 'initial'}
        )
        
        # Control buttons
        self.clear_button = Button(
            description='Clear All',
            button_style='warning',
            icon='trash'
        )
        self.clear_button.on_click(self._clear_all)
        
        self.copy_button = Button(
            description='Copy WKT',
            button_style='info',
            icon='copy'
        )
        self.copy_button.on_click(self._copy_wkt)
        
        # Load WKT functionality
        self.wkt_input = Textarea(
            placeholder='Paste WKT string here to visualize...',
            description='Load WKT:',
            layout=Layout(height='80px', width='100%'),
            style={'description_width': 'initial'}
        )
        
        self.load_button = Button(
            description='Load WKT',
            button_style='success',
            icon='upload'
        )
        self.load_button.on_click(self._load_wkt)
    
    def _handle_draw(self, target, action, geo_json: Dict[str, Any]) -> None:
        """
        Handle drawing events.
        
        Args:
            target: The draw control target
            action: The action performed
            geo_json (Dict): GeoJSON representation of the drawn feature
        """
        if geo_json['geometry']['type'] in ['Polygon', 'Rectangle']:
            self._add_polygon(geo_json)
            self._update_wkt_output()
            
            with self.output:
                print(f"✅ {geo_json['geometry']['type']} drawn successfully!")
    
    def _handle_data_change(self, change) -> None:
        """
        Handle changes in the draw control data (edits and deletions).
        
        Args:
            change: The change event containing new and old data
        """
        try:
            # Update our polygon list based on current draw control data
            current_data = change['new']
            
            # Clear current polygons and rebuild from draw control data
            self.polygons.clear()
            
            # Handle different data structures
            if isinstance(current_data, dict) and 'features' in current_data:
                # GeoJSON FeatureCollection format
                features = current_data['features']
            elif isinstance(current_data, list):
                # Direct list of features
                features = current_data
            else:
                # Unknown format, skip
                return
            
            # Process each feature
            for feature in features:
                if isinstance(feature, dict) and 'geometry' in feature:
                    if feature['geometry']['type'] in ['Polygon', 'Rectangle']:
                        self._add_polygon(feature)
            
            self._update_wkt_output()
            
            with self.output:
                if len(self.polygons) == 0:
                    print("🗑️ All polygons cleared!")
                else:
                    print(f"✏️ Polygons updated! Current count: {len(self.polygons)}")
                    
        except Exception as e:
            with self.output:
                print(f"⚠️ Error handling data change: {str(e)}")
    
    def _add_polygon(self, geo_json: Dict[str, Any]) -> None:
        """
        Add a polygon to the internal storage.
        
        Args:
            geo_json (Dict): GeoJSON representation of the polygon
        """
        polygon_data = {
            'id': len(self.polygons),
            'geo_json': geo_json,
            'coordinates': geo_json['geometry']['coordinates']
        }
        self.polygons.append(polygon_data)
    
    def _coordinates_to_wkt(self, coordinates: List[List[List[float]]]) -> str:
        """
        Convert polygon coordinates to WKT format.
        
        Args:
            coordinates (List): Polygon coordinates in GeoJSON format
            
        Returns:
            str: WKT representation of the polygon
        """
        # Handle both Polygon and Rectangle geometries
        if len(coordinates) > 0 and len(coordinates[0]) > 0:
            # Get the exterior ring coordinates
            exterior_coords = coordinates[0]
            
            # Ensure the polygon is closed (first and last points are the same)
            if exterior_coords[0] != exterior_coords[-1]:
                exterior_coords.append(exterior_coords[0])
            
            # Convert to WKT format (lon lat)
            coord_strings = [f"{lon} {lat}" for lon, lat in exterior_coords]
            return f"POLYGON(({', '.join(coord_strings)}))"
        
        return ""
    
    def _update_wkt_output(self) -> None:
        """
        Update the WKT output textarea with current polygons.
        """
        if not self.polygons:
            self.wkt_output.value = ""
            return
        
        wkt_strings = []
        for i, polygon in enumerate(self.polygons):
            wkt = self._coordinates_to_wkt(polygon['coordinates'])
            if wkt:
                wkt_strings.append(f"-- Polygon {i+1} --\n{wkt}")
        
        self.wkt_output.value = "\n\n".join(wkt_strings)
    
    def _clear_all(self, button) -> None:
        """
        Clear all drawn polygons.
        
        Args:
            button: The button widget that triggered this event
        """
        self.draw_control.clear()
        self.polygons.clear()
        self.wkt_output.value = ""
        
        with self.output:
            print("🧹 All polygons cleared!")
    
    def _copy_wkt(self, button) -> None:
        """
        Copy WKT to clipboard (display instruction).
        
        Args:
            button: The button widget that triggered this event
        """
        with self.output:
            print("📋 Select and copy the WKT text from the output area above.")
    
    def _load_wkt(self, button) -> None:
        """
        Load WKT string and display on map.
        
        Args:
            button: The button widget that triggered this event
        """
        wkt_string = self.wkt_input.value.strip()
        if not wkt_string:
            with self.output:
                print("❌ Please enter a WKT string to load.")
            return
        
        try:
            # Parse WKT string
            geometry = wkt.loads(wkt_string)
            
            if geometry.geom_type != 'Polygon':
                with self.output:
                    print(f"❌ Only Polygon geometries are supported. Got: {geometry.geom_type}")
                return
            
            # Convert to GeoJSON and add to map
            geo_json_feature = {
                'type': 'Feature',
                'geometry': mapping(geometry),
                'properties': {}
            }
            
            # Add as GeoJSON layer
            geojson_layer = GeoJSON(
                data=geo_json_feature,
                name=f"Loaded Polygon",
                style={
                    'fillColor': '#ffaa00',
                    'color': '#ff8800',
                    'fillOpacity': 0.3,
                    'weight': 2
                }
            )
            self.map.add_layer(geojson_layer)
            
            # Fit map to geometry bounds
            bounds = geometry.bounds  # (minx, miny, maxx, maxy)
            self.map.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
            
            with self.output:
                print(f"✅ WKT polygon loaded successfully!")
                
        except Exception as e:
            with self.output:
                print(f"❌ Error loading WKT: {str(e)}")
    
    def get_wkt_polygons(self) -> List[str]:
        """
        Get all drawn polygons as WKT strings.
        
        Returns:
            List[str]: List of WKT strings for all polygons
        """
        wkt_polygons = []
        for polygon in self.polygons:
            wkt = self._coordinates_to_wkt(polygon['coordinates'])
            if wkt:
                wkt_polygons.append(wkt)
        return wkt_polygons
    
    def display(self) -> VBox:
        """
        Display the complete polygon tool interface.
        
        Returns:
            VBox: The complete UI widget
        """
        # Instructions
        instructions = HTML(
            value="""
            <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                <h4>🗺️ How to use the Interactive Polygon Tool:</h4>
                <ul>
                    <li><strong>Draw:</strong> Use the polygon or rectangle tools on the map</li>
                    <li><strong>Edit:</strong> Click on a drawn polygon and drag the vertices</li>
                    <li><strong>Delete:</strong> Select a polygon and use the delete tool</li>
                    <li><strong>Export:</strong> Copy the WKT coordinates for use in searches</li>
                    <li><strong>Import:</strong> Load existing WKT strings to visualize</li>
                </ul>
            </div>
            """
        )
        
        # Button row
        button_row = HBox([self.clear_button, self.copy_button])
        
        # Load WKT section
        load_section = VBox([
            HTML("<h4>📥 Load WKT Polygon</h4>"),
            self.wkt_input,
            self.load_button
        ])
        
        # Output section
        output_section = VBox([
            HTML("<h4>📤 WKT Output</h4>"),
            self.wkt_output,
            button_row
        ])
        
        return VBox([
            instructions,
            self.map,
            load_section,
            output_section,
            HTML("<h4>📋 Messages</h4>"),
            self.output
        ])

print("✅ InteractivePolygonTool class created successfully!")

✅ InteractivePolygonTool class created successfully!


## Create and Display the Interactive Tool

Now let's create an instance of our polygon tool and display it:

In [3]:
# Create the interactive polygon tool
# You can customize the initial location and zoom level
polygon_tool = InteractivePolygonTool(
    center=(45.0, 0.0),  # Initial center (latitude, longitude)
    zoom=3,  # Initial zoom level
    basemap=basemaps.OpenStreetMap.Mapnik  # You can change the basemap
)

# Display the tool
display(polygon_tool.display())

## Using the Tool with phidown

Once you've drawn polygons, you can use them directly with the phidown library for satellite data searches:

In [5]:
# Example of how to use with phidown (uncomment to use)
import sys
from phidown.search import CopernicusDataSearcher

wkt_polygons = polygon_tool.get_wkt_polygons()
    
if not wkt_polygons:
    print("❌ No polygons drawn yet. Please draw a polygon on the map first.")
    sys.exit(1)

print(f"📍 Found {len(wkt_polygons)} polygon(s):")

for i, wkt_polygon in enumerate(wkt_polygons):
    print(f"\nPolygon {i+1}:")
    print(f"WKT: {wkt_polygon}")


    searcher = CopernicusDataSearcher()
    searcher._query_by_filter(
        collection_name='SENTINEL-2',
        product_type="S2MSI1C",
        aoi_wkt=wkt_polygon,  # Use the drawn polygon as AOI
        start_date='2023-06-01T00:00:00',
        end_date='2023-06-03T00:00:00',
        top=10
    )
    df = searcher.execute_query()
    print(f"Found {len(df)} products for this polygon")
    # save the results to a CSV file
    df.to_csv(f"sentinel2_products_polygon_{i+1}.csv", index=False)
    print(f"Results saved to sentinel2_products_polygon_{i+1}.csv")


📍 Found 1 polygon(s):

Polygon 1:
WKT: POLYGON((14.073628 40.907626, 14.33725 40.908664, 14.587141 40.754881, 14.615975 40.620549, 14.33313 40.528757, 13.810006 40.591356, 13.742727 40.774643, 14.073628 40.907626))
Found 8 products for this polygon
Results saved to sentinel2_products_polygon_1.csv
Found 8 products for this polygon
Results saved to sentinel2_products_polygon_1.csv


## Advanced Features

### Custom Styled Map with Satellite Imagery

In [7]:
# Create a tool with satellite imagery basemap
satellite_tool = InteractivePolygonTool(
    center=(37.7749, -122.4194),  # San Francisco coordinates
    zoom=10,
    basemap=basemaps.Esri.WorldImagery  # Satellite imagery
)

print("🛰️ Satellite imagery polygon tool created!")
print("Uncomment the line below to display it:")
display(satellite_tool.display())

🛰️ Satellite imagery polygon tool created!
Uncomment the line below to display it:


### Example: Load a Predefined Polygon

Let's demonstrate loading a WKT polygon (example from Sicily, Italy):

In [8]:
# Example WKT polygon for Sicily area (from the original notebook)
sicily_wkt = "POLYGON((14.889908 37.722392, 14.960632 37.672408, 15.113068 37.695231, 15.109634 37.804359, 14.956512 37.827684, 14.889908 37.722392))"

# Create a tool centered on Sicily
sicily_tool = InteractivePolygonTool(
    center=(37.75, 14.98),  # Sicily coordinates
    zoom=10,
    basemap=basemaps.OpenStreetMap.Mapnik
)

print(f"🇮🇹 Sicily polygon tool created!")
print(f"Example WKT for Sicily: {sicily_wkt}")
print("\nTo load this polygon:")
print("1. Display the tool using: display(sicily_tool.display())")
print("2. Paste the WKT string in the 'Load WKT' text area")
print("3. Click 'Load WKT' button")

# Uncomment to display:
# display(sicily_tool.display())

🇮🇹 Sicily polygon tool created!
Example WKT for Sicily: POLYGON((14.889908 37.722392, 14.960632 37.672408, 15.113068 37.695231, 15.109634 37.804359, 14.956512 37.827684, 14.889908 37.722392))

To load this polygon:
1. Display the tool using: display(sicily_tool.display())
2. Paste the WKT string in the 'Load WKT' text area
3. Click 'Load WKT' button


## Integration Example with phidown

Here's a complete example showing how to integrate the polygon tool with a phidown search:

In [9]:
def search_with_drawn_polygon():
    """
    Complete example of using drawn polygons for satellite data search.
    """
    from phidown.search import CopernicusDataSearcher
    import pandas as pd
    
    # Get WKT polygons from the tool
    wkt_polygons = polygon_tool.get_wkt_polygons()
    
    if not wkt_polygons:
        print("❌ Please draw a polygon first using the tool above.")
        return None
    
    # Use the first polygon for search
    aoi_wkt = wkt_polygons[0]
    print(f"🔍 Searching with polygon: {aoi_wkt[:100]}...")
    
    try:
        # Configure search
        searcher = CopernicusDataSearcher()
        searcher._query_by_filter(
            collection_name='SENTINEL-2',
            product_type="S2MSI1C",
            orbit_direction=None,
            cloud_cover_threshold=20,  # Max 20% cloud cover
            aoi_wkt=aoi_wkt,  # Use the drawn polygon
            start_date='2024-05-01T00:00:00',
            end_date='2024-05-31T00:00:00',
            top=5  # Limit to 5 results for demo
        )
        
        # Execute search
        df = searcher.execute_query()
        
        if len(df) > 0:
            print(f"✅ Found {len(df)} Sentinel-2 products!")
            searcher.display_results(top_n=3)
            return df
        else:
            print("❌ No products found for the specified criteria.")
            return None
            
    except Exception as e:
        print(f"❌ Error during search: {str(e)}")
        return None

print("🔧 Search function ready!")
print("After drawing a polygon, run: search_with_drawn_polygon()")

🔧 Search function ready!
After drawing a polygon, run: search_with_drawn_polygon()


In [10]:
search_with_drawn_polygon()

🔍 Searching with polygon: POLYGON((14.073628 40.907626, 14.33725 40.908664, 14.587141 40.754881, 14.615975 40.620549, 14.33313...
✅ Found 5 Sentinel-2 products!
✅ Found 5 Sentinel-2 products!


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes
0,application/octet-stream,2e17af6c-22ab-44e7-959e-61e13739fcdf,S2B_MSIL1C_20240526T094549_N0510_R079_T33TUF_2...,application/octet-stream,285544457,2024-05-26 11:39:43,2024-05-26T11:48:26.185466Z,2024-11-12T15:12:17.142821Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2024/05/26/S2B_MSIL...,"[{'Value': '904d6e87ecdfd022b1b3321623709b12',...","{'Start': '2024-05-26T09:45:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((13.2267941866508...,"{'type': 'Polygon', 'coordinates': [[[13.22679...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,application/octet-stream,40f370a1-067d-48a8-8fcc-6a202a3ef6ff,S2B_MSIL1C_20240526T094549_N0510_R079_T33TVE_2...,application/octet-stream,744392447,2024-05-26 11:48:11,2024-05-26T11:59:00.722712Z,2024-11-12T15:12:08.434881Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2024/05/26/S2B_MSIL...,"[{'Value': '52e4a5fc11d2304212a9eb452ef7e5d7',...","{'Start': '2024-05-26T09:45:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((13.8168273933968...,"{'type': 'Polygon', 'coordinates': [[[13.81682...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,application/octet-stream,8dcf6799-f1d7-453a-86f1-ad75b826dc84,S2B_MSIL1C_20240526T094549_N0510_R079_T33TUE_2...,application/octet-stream,482010864,2024-05-26 11:46:48,2024-05-26T11:55:22.248188Z,2024-11-12T15:11:14.655509Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2024/05/26/S2B_MSIL...,"[{'Value': '5911b0c6400bd7a75fe92cccd861fca5',...","{'Start': '2024-05-26T09:45:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((12.9535931316802...,"{'type': 'Polygon', 'coordinates': [[[12.95359...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,application/octet-stream,e8a51143-311d-4c8a-86bf-addfe6c92266,S2B_MSIL1C_20240526T094549_N0510_R079_T33TVF_2...,application/octet-stream,837161753,2024-05-26 11:56:06,2024-05-26T12:03:47.572396Z,2024-11-12T15:10:20.951879Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2024/05/26/S2B_MSIL...,"[{'Value': '82180f91cc0181ebcdf3938b33f83cbd',...","{'Start': '2024-05-26T09:45:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((13.8005508983666...,"{'type': 'Polygon', 'coordinates': [[[13.80055...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
4,application/octet-stream,eb23a921-5b28-4b75-b372-2dea9877d554,S2A_MSIL1C_20240524T100031_N0510_R122_T33TUF_2...,application/octet-stream,696520769,2024-05-24 15:55:18,2024-05-24T16:09:25.063340Z,2024-05-24T16:18:01.538850Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2024/05/24/S2A_MSIL...,"[{'Value': '310fdb4802c4ebb28883f0af528b9301',...","{'Start': '2024-05-24T10:00:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((12.6028173807795...,"{'type': 'Polygon', 'coordinates': [[[12.60281...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
